In [1]:
import json
import os
import matplotlib.pyplot as plt
import glob
from openai import OpenAI
import requests
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import glob
import os

AttributeError: module 'pydantic_core' has no attribute 'tuple_schema'

In [ ]:
from r2r.client import R2RClient
from r2r.core.utils import generate_id_from_label

In [1]:
#load openAI key from .env
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

NameError: name 'os' is not defined

In [ ]:


# Initialize the client with the base URL of your API
base_url = "https://sciphi-0d04df5c-7e2c-43e2-be14-a320adb1676f-qwpin2swwa-ue.a.run.app"  # Change this to your actual API base URL
client = R2RClient(base_url)

user_id_0 = generate_id_from_label("user_0")



In [ ]:
prompt = """You are given a user query {query} and a user context {context}. Use the context to answer the given query. """
formatted_prompt = prompt.format(
    query="What are the key themes of these books?",
    context="User Uploads:\nContexts Pqau Split Part 1.pdf",
)

# Perform a search on this file
search_response = client.rag_completion(
    formatted_prompt, 5, filters={"user_id": user_id_0}
)

In [45]:
def prompt(payload):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    }

    try:
        resp = requests.post("https://api.openai.com/v1/chat/completions", 
                             headers=headers, json=payload)
        data = resp.json()
        resp_dict = data['choices'][0]['message']['content']

        return resp_dict
    
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as err:
        print(f"An error occurred: {err}")
    except KeyError as key_err:
        print(f"Key error: {key_err} - the structure of the response might not be as expected.")
    
    return None


### Load data

In [22]:
data = json.load(open('pubmedqa-master/data/test_set.json'))
labels = json.load(open('pubmedqa-master/data/test_ground_truth.json'))
len(data), len(labels)

(500, 500)

In [ ]:
predictions, gt = {}, {}

class_dict = { 'no': 0, 'yes': 1, 'maybe': 2}

for i, q in data.items():
    text_prompt = "Answer question given user query context: {} . Answer can only be one word and it should be either 'yes', 'no', or 'maybe'.".format(str(q))

    prompt_data = {
        "model": 'gpt-4-vision-preview', 
        "temperature": 0,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": text_prompt},
                ],
            }
        ],
        "max_tokens": f"{30}",
    }
    resp = prompt(prompt_data)
    
    predictions.update({i: class_dict[resp.lower()]})
    gt.update({i: class_dict[labels[i]]})


In [ ]:
len(predictions), len(gt)

In [ ]:
accuracy_score(list(gt.values()), list(predictions.values() ))

In [ ]:
f1_score(list(gt.values()), list(predictions.values()), average='macro')

### GPT3.5

In [48]:
predictions, gt = {}, {}

class_dict = { 'no': 0, 'yes': 1, 'maybe': 2}

for i, q in data.items():
    text_prompt = "Answer question given context: {} . Answer can only be one word and it should be either 'yes', 'no', or 'maybe'.".format(str(q))

    prompt_data = {
        "model": 'gpt-3.5-turbo', 
        "temperature": 0,
        "messages": [
            {
        "role": "system",
        "content": "You are a helpful medical assistant."
      },
            {
                "role": "user",
                "content": text_prompt
            }
        ]
    }
    resp = prompt(prompt_data)

    predictions.update({i: class_dict[resp.lower()]})
    gt.update({i: class_dict[labels[i]]})


In [49]:
len(predictions), len(gt)

(500, 500)

In [55]:
accuracy_score(list(gt.values()), list(predictions.values() ))

0.928

In [60]:
f1_score(list(gt.values()), list(predictions.values()), average='macro')

0.8875015232868346